# Simple Index Demo

#### Load documents, build the GPTSimpleVectorIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import (
    GPTSimpleVectorIndex, 
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext
)
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from IPython.display import Markdown, display

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# LLM Predictor (gpt-3)
llm_predictor_gpt3 = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))
service_context_gpt3 = ServiceContext.from_defaults(llm_predictor=llm_predictor_gpt3)

# LLMPredictor (gpt-4)
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context_gpt4 = ServiceContext.from_defaults(llm_predictor=llm_predictor_gpt4)

In [3]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [4]:
index = GPTSimpleVectorIndex.from_documents(documents)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens
> [build_index_from_nodes] Total embedding token usage: 17617 tokens


In [5]:
# save index to disk
index.save_to_disk('index_simple.json')

In [6]:
# load index from disk
index = GPTSimpleVectorIndex.load_from_disk('index_simple.json')

#### Query Index

In [7]:
from gpt_index.indices.query.query_transform.base import StepDecomposeQueryTransform
# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(
    llm_predictor_gpt4, verbose=True
)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm_predictor_gpt3, verbose=True
)

In [8]:
index.index_struct.summary = "Used to answer questions about the author"

In [9]:
# set Logging to DEBUG for more detailed outputs
response_gpt4 = index.query(
    "Who was in the first batch of the accelerator program the author started?",
    query_transform=step_decompose_transform,
    service_context=service_context_gpt4
)

> Current query: Who was in the first batch of the accelerator program the author started?
> Formatted prompt: The original question is as follows: Who was in the first batch of the accelerator program the author started?
We have an opportunity to answer some, or all of the question from a knowledge source. Context information for the knowledge source is provided below, as well as previous reasoning steps.
Given the context and previous reasoning, return a question that can be answered from the context. This question can be the same as the original question, or this question can represent a subcomponent of the overall question.It should not be irrelevant to the original question.
If we cannot extract more information from the context, provide 'None' as the answer. Some examples are given below: 

Question: How many Grand Slam titles does the winner of the 2020 Australian Open have?
Knowledge source context: Provides names of the winners of the 2020 Australian Open
Previous reasoning: N

In [10]:
display(Markdown(f"<b>{response_gpt4}</b>"))

<b>

The participants in the first batch of Y Combinator included the founders of Reddit (Steve Huffman and Alexis Ohanian), Loopt (Sam Altman, Nick Sivo, and Daniel Gross), Weebly (David Rusenko, Chris Fanini, and Dan Veltri), Justin Kan and Emmett Shear (who later founded Twitch), Aaron Swartz (who had helped write the RSS spec and later became a martyr for open access), and Sam Altman (who later became the second president of YC).</b>

In [11]:
sub_qa = response_gpt4.extra_info["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[('What accelerator program did the author start?', 'The author started the accelerator program Y Combinator.'), ('Who was in the first batch of Y Combinator?', 'The first batch of Y Combinator included the founders of Reddit, Justin Kan and Emmett Shear (who later founded Twitch), Aaron Swartz, and Sam Altman.'), ('Who were the participants in the first batch of Y Combinator?', 'The participants in the first batch of Y Combinator included the founders of Reddit, Justin Kan and Emmett Shear (who later founded Twitch), Aaron Swartz (who had helped write the RSS spec and later became a martyr for open access), and Sam Altman (who later became the second president of YC).')]


In [13]:
response_gpt4 = index.query(
    "In which city did the author found his first company, Viaweb?",
    query_transform=step_decompose_transform,
    service_context=service_context_gpt4
)

> Current query: In which city did the author found his first company, Viaweb?
> Formatted prompt: The original question is as follows: In which city did the author found his first company, Viaweb?
We have an opportunity to answer some, or all of the question from a knowledge source. Context information for the knowledge source is provided below, as well as previous reasoning steps.
Given the context and previous reasoning, return a question that can be answered from the context. This question can be the same as the original question, or this question can represent a subcomponent of the overall question.It should not be irrelevant to the original question.
If we cannot extract more information from the context, provide 'None' as the answer. Some examples are given below: 

Question: How many Grand Slam titles does the winner of the 2020 Australian Open have?
Knowledge source context: Provides names of the winners of the 2020 Australian Open
Previous reasoning: None
Next question: Who w

In [14]:
print(response_gpt4)



Answer: Paul Graham founded his first company, Viaweb, in Cambridge, Massachusetts.


In [15]:
response_gpt3 = index.query(
    "In which city did the author found his first company, Viaweb?",
    query_transform=step_decompose_transform_gpt3,
    service_context=service_context_gpt3
)

> Current query: In which city did the author found his first company, Viaweb?
> Formatted prompt: The original question is as follows: In which city did the author found his first company, Viaweb?
We have an opportunity to answer some, or all of the question from a knowledge source. Context information for the knowledge source is provided below, as well as previous reasoning steps.
Given the context and previous reasoning, return a question that can be answered from the context. This question can be the same as the original question, or this question can represent a subcomponent of the overall question.It should not be irrelevant to the original question.
If we cannot extract more information from the context, provide 'None' as the answer. Some examples are given below: 

Question: How many Grand Slam titles does the winner of the 2020 Australian Open have?
Knowledge source context: Provides names of the winners of the 2020 Australian Open
Previous reasoning: None
Next question: Who w

In [16]:
print(response_gpt3)



The author founded his first company, Viaweb, in Cambridge, Massachusetts in 1993.
